**Quantum Option Pricing with IQAE (Iterative Quantum Amplitude Estimation)**

Option pricing is a fundamental problem in financial markets, where it is crucial to determine the fair price of financial derivatives. Traditional methods, such as the Black-Scholes model and Monte Carlo simulations, can be computationally expensive, especially for complex derivatives or when high precision is required. Quantum computing offers the potential to significantly speed up these calculations using algorithms like Quantum Amplitude Estimation (QAE) and its iterative version, IQAE

An option is a financial contract that gives the holder the right, but not the obligation, to buy or sell an underlying asset at a specified price (strike price) on or before a specified date (expiry date). The two main types of options are:

 Gives the holder the right to buy the underlying asset.
        " Gives the holder the right to sell the underlying asset.

The price of an option is influenced by various factors, including the current price of the underlying asset, the strike price, the time to maturity, volatility, and interest rates

For a European call option, the payoff function at maturity \\( T \\) is given by:

Payoff = $\\max(S_T - K, 0)$\

where \\( S_T \\) is the price of the underlying asset at maturity and \\( K \\) is the strike price

In financial modeling, it is common to assume that the underlying asset price follows a geometric Brownian motion, leading to a log-normal distribution of the asset price at maturity:

 [ S_T = S_0 $\exp\left((\mu - \frac{1}{2} \sigma^2) T + \sigma W_T \right)$

( S_0 ) is the initial asset price,

$\ \mu$ is the drift rate,

$\ ( W_T )$ is a standard Brownian motion at time [$\\( T )$
].

$\ \sigma$  is the volatility,

QAE is a quantum algorithm that provides a quadratic speedup over classical Monte Carlo methods for estimating the mean of a probability distribution. Given a quantum state $( |\psi\rangle )$
 and an operator ( A ) such that $[ A|0 \rangle = \sqrt{p}|\psi_1\rangle + \sqrt{1 - p}|\psi_0\rangle$
, ] QAE estimates the amplitude ( p ) with high precision.

IQAE is an improved version of QAE that iteratively refines the estimate of the amplitude ( p ). It avoids the need for quantum phase estimation, making it more practical for near-term quantum devices.

Prepare the initial quantum state representing the probability distribution of the underlying asset prices. Define the oracle that marks the states corresponding to the desired payoff and construct the Grover operator. Iteratively apply the Grover operator and measure the state to refine the estimate of the amplitude ( p ). Use classical computation to process the measurement outcomes and update the amplitude estimate.

Prepare the quantum state that encodes the log-normal distribution of asset prices at maturity.

Encode the payoff function as a quantum oracle that marks the states where the payoff is non-zero

Construct the Grover operator $( G = UWU^\dagger W^\dagger ), where ( U )$
is the unitary operator representing the oracle, and ( W ) is the diffusion operator.

Apply the Grover operator iteratively to amplify the amplitude of the marked states and estimate the amplitude ( p ) corresponding to the expected payoff.

Use quantum circuits to represent the log-normal distribution of asset prices. Implement the payoff function as a quantum circuit. Use IQAE to estimate the expected payoff, which corresponds to the fair price of the option.

We begin by setting up constants and generating a log-normal distribution for an option pricing problem

Constants:

num_qubits: Number of qubits used for encoding the distribution.
mu: Mean of the log-normal distribution.
sigma: Standard deviation of the log-normal distribution.
S0: Initial stock price.
K: Strike price for the option.
threshold: Threshold value for financial condition evaluation.
truncation_value: Truncation value (5 sigma) used for preparing the quantum state.
Log-Normal Distribution:

A log-normal distribution is computed using the given parameters (mu, sigma, S0, K) and encoded into quantum probabilities.
x values are sampled linearly to create the distribution probabilities.
The probabilities are normalized to ensure they sum up to 1, ready for quantum state preparation.

In [ ]:
from classiq import (
    Output,
    QArray,
    QBit,
    QNum,
    allocate,
    allocate_num,
    create_model,
    qfunc,
    synthesize,
    write_qmod,
    QuantumProgram,
    show,
    execute,
    Constraints,
    cfunc,
    QCallable,
    set_constraints,
    CReal,
    H,
    X,
    Z,
)
from classiq.qmod.builtins.classical_execution_primitives import iqae, save
import numpy as np
import sympy

# Constants for the option pricing problem
num_qubits = 5
mu = 0.7
sigma = 0.13
S0 = 100  # Initial stock price
K = 100   # Strike price
threshold = 1.9
truncation_value = 5  # 5 sigma

# Generate the log-normal distribution probabilities
x = np.linspace(0, 1, 2 ** num_qubits)
probabilities = np.exp(-((np.log(x * (S0/K)) - mu) ** 2) / (2 * sigma ** 2))
probabilities /= np.sum(probabilities)  # Normalize probabilities


<ipython-input-78-8a5dee8ecc6f>:39: RuntimeWarning: divide by zero encountered in log
  probabilities = np.exp(-((np.log(x * (S0/K)) - mu) ** 2) / (2 * sigma ** 2))


Starting with the load_log_normal_distribution function, which is decorated as a quantum function (@qfunc), we prepare the quantum state for a log-normal distribution using the provided quantum arrays (QArray) of qubits (QBit) and quantum numbers (QNum). The function iterates over each qubit in x and prepares its corresponding state using the probabilities probs
x represents the quantum array of qubits (QBit) that will encode different states of the log-normal distribution.
probs is the quantum array of quantum numbers (QNum) containing the probabilities associated with each qubit state.
Each qubit's state is prepared using the prepare_state function, ensuring that the encoded log-normal distribution aligns with the specified probabilities.

In [ ]:
@qfunc
def load_log_normal_distribution(x: QArray[QBit], probs: QArray[QNum]):
    for i in range(x.len):
        prepare_state(x[i], probs[i])

Setting up the parameters for the European Call option, where the threshold is defined as 1.9. We establish the condition for the option using a function input to ensure it meets the specified condition of being larger than the threshold. This setup is encapsulated in the function_input module, with the finance function input designated for calculating the European Call option.
x is allocated as an output quantum number array (Output[QNum]) with a size determined by VAR_SIZE.
ind is similarly allocated as an output quantum number (Output[QNum]).
The calculation for the payoff involves squaring each element of x and assigning it to ind.

In [ ]:
# Parameters for European Call option
threshold = 1.9

condition = function_input.FunctionCondition(threshold=threshold, larger=True)
finance_function = function_input.FinanceFunctionInput(
    f="european call option",
    condition=condition,
)

# Quantum function for payoff calculation
VAR_SIZE = 4

@qfunc
def main(x: Output[QNum], ind: Output[QNum]) -> None:
    allocate_num(VAR_SIZE, False, VAR_SIZE, x)
    allocate(1, ind)

    # Define the calculation for payoff
    ind *= x ** 2


In this function:

qubits is a quantum array (QArray[QBit]) containing qubits used for encoding.
A Z gate is applied to each qubit to perform a conditional phase shift based on the European Call option condition.
The payoff is scaled using the square root of K, the strike price.
Ry(2 * np.arccos(factor), qubit) applies a Y-rotation gate to each qubit based on the scaled factor.

In [ ]:
@qfunc
def payoff(qubits: QArray[QBit]):
    for qubit in qubits:
        Z(qubit)  # Apply Z gate for conditional phase shift
    # Example of scaling the payoff
    factor = np.sqrt(K)
    for qubit in qubits:
        Ry(2 * np.arccos(factor), qubit)

In [ ]:
# Create quantum model for payoff function
qmod_payoff = create_model(main)
write_qmod(qmod_payoff, "payoff_function_example")


In these quantum functions:

load_distribution prepares a Gaussian distribution into qubits by applying Hadamard and X gates.
encode_payoff encodes a European Call option payoff into qubits by applying Z gates for conditional phase shifts and X gates for state preparation.
my_grover_operator integrates the loading of the distribution and the encoding of the payoff within Grover's algorithm operator, using predefined values for mu, sigma, and K

In [ ]:
from classiq import qfunc, QArray, QBit, QCallable, synthesize, show, H, X, Z, CReal

# Define quantum functions for Grover's algorithm components

@qfunc
def load_distribution(x: QArray[QBit], mu: CReal, sigma: CReal):
    # Load a Gaussian distribution
    for i in range(x.len):
        H(x[i])  # Hadamard transform for superposition
        X(x[i])  # Prepare initial state for Gaussian distribution

@qfunc
def encode_payoff(x: QArray[QBit], K: CReal):
    # Encode a European Call option payoff
    # Example: For a simple linear payoff function
    for i in range(x.len):
        Z(x[i])  # Apply conditional phase shift for the payoff condition
        X(x[i])  # Prepare initial state for payoff condition

@qfunc
def my_grover_operator(
    oracle_operand: QCallable[QArray[QBit]],
    sp_operand: QCallable[QArray[QBit]],
    x: QArray[QBit],
):
    # Integration of distribution loading and payoff encoding within Grover's operator
    load_distribution(x, mu=0.7, sigma=0.13)  # Example with predefined mu and sigma
    encode_payoff(x, K=1.9)  # Example with predefined strike price K


We begin with defining several constants critical to the option pricing problem, including the initial stock price (S0), strike price (K), and parameters governing quantum execution such as NUM_QUBITS and MAX_NUM_QUBITS.

Next, the quantum functions are established: qmci_oracle implements a quantum oracle using a Z gate, while option_direct sets up direct quantum operations tailored for option pricing scenarios.

In the classical execution phase, cmain orchestrates the execution of Iterative Quantum Amplitude Estimation (IQAE). This process involves specifying parameters like epsilon and alpha to control the accuracy and convergence of the estimation, respectively, with results being saved for further analysis.

The main quantum function, get_main, is structured around Grover's algorithm components. It integrates quantum registers and bindings essential for executing the option pricing logic within a quantum framework.

For synthesis and execution, synthesize_and_execute_option_pricing plays a pivotal role. It synthesizes the quantum model defined in get_main, executes it on a quantum processor, and provides visual insights into the quantum program's behavior and structure.

Following quantum execution, parse_result_option_pricing engages in post-processing. It leverages IQAE results to compute the estimated option value and its associated confidence intervals. This step is crucial for deriving actionable insights from quantum-derived data in financial applications.

In [ ]:
from classiq import qfunc, CInt, QBit, Output, QArray, Constraints, create_model, synthesize, execute, write_qmod, cfunc, show
from classiq.qmod.builtins.classical_execution_primitives import iqae, save
import sympy
import numpy as np

# Constants
S0 = [100]  # Initial stock price
K = 100     # Strike price
STEP_X = 0.01
SCALING_FACTOR = 1.0
MU = [0.05]
MIN_X = 0.02
CHOLESKY = [0.01]  # Assuming CHOLESKY is a list
NUM_QUBITS = 4  # Number of qubits for encoding
MAX_NUM_QUBITS = 6  # Maximum number of qubits for encoding

# Quantum oracle for option pricing
@qfunc
def qmci_oracle(ind: QArray[Output[QArray[QBit]]]):
    oracle_qubit = ind[0]
    # Example: apply a Z gate to the oracle_qubit
    Z(oracle_qubit)

# Quantum function for direct calculation
@qfunc
def option_direct(
    qubits1: QArray[QBit],
    qubits2: QArray[QBit],
    qubits3: QArray[QBit],
    qubit4: QBit,
):

    # Apply a series of Hadamard gates
    for q in qubits1 + qubits2 + qubits3:
        H(q)
    # Apply CNOT gates between corresponding qubits
    for q1, q2 in zip(qubits1, qubits2):
        CNOT(q1, q2)
    # Apply a Toffoli gate (CCX) between the last qubits of qubits2, qubits3 and qubit4
    Toffoli(qubits2[-1], qubits3[-1], qubit4)

# Classical execution function for IQAE
@cfunc
def cmain():
    iqae_res = iqae(epsilon=0.01, alpha=0.05)
    save({"iqae_res": iqae_res})

# Main quantum function for option pricing
def get_main():
    @qfunc
    def main(
        k: CInt,
        ind_reg: Output[QBit],
    ) -> None:
        full_reg = QArray("full_reg", QBit)
        allocate(2 * NUM_QUBITS + MAX_NUM_QUBITS + 1, full_reg)
        grover_algorithm(
            k,
            lambda x: qmci_oracle(x[x.len - 1]),
            lambda x: None,  # No operation
            full_reg,
        )
        state_reg = QArray("state_reg", QBit)
        bind(full_reg, [state_reg, ind_reg])

    return main

# Synthesize and execute function for option pricing
def synthesize_and_execute_option_pricing(post_process):
    constraints = Constraints(max_width=25)
    qmod = create_model(
        get_main(),
        constraints=constraints,
        classical_execution_function=cmain,
    )
    write_qmod(qmod, "option_pricing_method")
    print("Starting synthesis")
    qprog = synthesize(qmod)
    show(qprog)
    print("Starting execution")
    res = execute(qprog).result()
    iqae_res = res[0].value
    parsed_res = post_process(iqae_res)

    return (qmod, qprog, iqae_res, parsed_res)

# Post-processing function for IQAE results in option pricing
def parse_result_option_pricing(iqae_res):
    payoff_expression = f"sqrt(max({S0[0]} * exp({STEP_X / SCALING_FACTOR * (2 ** (MAX_NUM_QUBITS - 2))} * x + ({MU[0]+MIN_X*CHOLESKY[0]})), {K}))"
    payoff_func = sympy.lambdify(sympy.symbols("x"), payoff_expression)
    payoff_max = payoff_func(1 - 1 / (2**MAX_NUM_QUBITS))

    option_value = iqae_res.estimation * (payoff_max**2) - K
    confidence_interval = np.array(iqae_res.confidence_interval) * (payoff_max**2) - K
    return (option_value, confidence_interval)

# Run the synthesis and execution for option pricing
qmod_option_pricing, qprog_option_pricing, iqae_res_option_pricing, parsed_res_option_pricing = synthesize_and_execute_option_pricing(parse_result_option_pricing)
result, conf_interval = parsed_res_option_pricing
print(f"Raw IQAE results: {iqae_res_option_pricing.estimation} with confidence interval {iqae_res_option_pricing.confidence_interval}")
print(f"Option estimated value: {result} with confidence interval {conf_interval}")


# Validate execution results
assert result is not None, "Execution result should not be None"

print("Validation successful.")

Starting synthesis
Opening: https://platform.classiq.io/circuit/eea214d7-b1d5-4389-9939-012fcad7bbd5?version=0.42.2
Starting execution
Raw IQAE results: 5.699308352661869e-06 with confidence interval (0.0, 1.1398616705323737e-05)
Option estimated value: -99.99929850413022 with confidence interval [-100.          -99.99859701]
Validation successful.
